Reading csv file into a dataframe

In [3]:
import pandas as pd
df = pd.read_csv("academy_awards.csv", encoding = "ISO-8859-1")
print(df.head())

          Year               Category          Nominee  \
0  2010 (83rd)  Actor -- Leading Role    Javier Bardem   
1  2010 (83rd)  Actor -- Leading Role     Jeff Bridges   
2  2010 (83rd)  Actor -- Leading Role  Jesse Eisenberg   
3  2010 (83rd)  Actor -- Leading Role      Colin Firth   
4  2010 (83rd)  Actor -- Leading Role     James Franco   

                          Additional Info Won? Unnamed: 5 Unnamed: 6  \
0                      Biutiful {'Uxbal'}   NO        NaN        NaN   
1           True Grit {'Rooster Cogburn'}   NO        NaN        NaN   
2  The Social Network {'Mark Zuckerberg'}   NO        NaN        NaN   
3    The King's Speech {'King George VI'}  YES        NaN        NaN   
4              127 Hours {'Aron Ralston'}   NO        NaN        NaN   

  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  
0        NaN        NaN        NaN         NaN  
1        NaN        NaN        NaN         NaN  
2        NaN        NaN        NaN         NaN  
3        NaN        Na

Data Filtering

In [4]:
# extracting just first four digits from year data
df["Year"] = df["Year"].str[0:4]

In [5]:
# converting Year values to integer
df["Year"] = df["Year"].astype("int64") 

In [6]:
# creating a list containing four award categories
award_categories = ["Actor -- Leading Role",
                   "Actor -- Supporting Role",
                   "Actress -- Leading Role",
                   "Actress - Supporting Role"]

In [8]:
# considering data with Year after 2000
after_2000 = df[df["Year"] > 2000]

In [9]:
# data with Year > 2000 and belonging to categories 
# presnt in above list.
nominations = after_2000[after_2000["Category"].
                         isin(award_categories)]

In [14]:
import warnings
warnings.filterwarnings(action='ignore')

In [15]:
# Replacing NO with 0 and YES with 1 in column Won?
replace_with = {'No' : 0, 'YES' : 1}

nominations["Won?"] = nominations["Won?"].map(replace_with)


In [16]:
# Creating new column Won that has values from column 
# Won?
# Won? will be dropped later
nominations["Won"] = nominations["Won?"]

In [17]:
# drop Won? and all Unnamed columns and assigning the 
# resulting dataframe to 'final_nominations'.

# creating a list of columns to be dropped
drop_columns = ["Won?",
               "Unnamed: 5",
               "Unnamed: 6",
               "Unnamed: 7",
               "Unnamed: 8",
               "Unnamed: 9",
               "Unnamed: 10"]

final_nominations = nominations.drop(drop_columns, axis =1)

In [18]:
# Splitting information in 'Additional Info' column into 
# two separate columns for movie and charater name.
additional_info_one = final_nominations["Additional Info"].str.rstrip("'}")
additional_info_two = additional_info_one.str.split("{'")

movie_name = additional_info_two.str[0]
character_name = additional_info_two.str[1]

final_nominations["Movie"] = movie_name
final_nominations["Character"] = character_name

final_nominations = final_nominations.drop("Additional Info", axis=1)
final_nominations.head()

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,NaN,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,NaN,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,NaN,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,NaN,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,NaN,127 Hours,Aron Ralston


In [ ]:
# Exporting cleaned dataframe to SQLite
import sqlite3
conn = sqlite3.connect("nominations.db")
final_nominations.to_sql("nominations", conn, index=False)

In [24]:
# Verifying with SQL
query1 = "pragma table_info(nominations);"
query2 = "SELECT * FROM nominations LIMIT 5;"

print(conn.execute(query1).fetchall())
print(conn.execute(query2).fetchall())
conn.close()

[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Nominee', 'TEXT', 0, None, 0), (3, 'Won', 'REAL', 0, None, 0), (4, 'Movie', 'TEXT', 0, None, 0), (5, 'Character', 'TEXT', 0, None, 0)]
[(2010, 'Actor -- Leading Role', 'Javier Bardem', None, 'Biutiful ', 'Uxbal'), (2010, 'Actor -- Leading Role', 'Jeff Bridges', None, 'True Grit ', 'Rooster Cogburn'), (2010, 'Actor -- Leading Role', 'Jesse Eisenberg', None, 'The Social Network ', 'Mark Zuckerberg'), (2010, 'Actor -- Leading Role', 'Colin Firth', None, "The King's Speech ", 'King George VI'), (2010, 'Actor -- Leading Role', 'James Franco', None, '127 Hours ', 'Aron Ralston')]
